# ML-7. Прогнозирование биологического ответа (HW-3)

### Задание:
Обучить две модели; гипепараметры подобраны при помощи одного метода
Обучить две модели; гипепараметры подобраны при помощи двух методов
Обучить две модели; гипепараметры подобраны при помощи трёх методов
Обучить две модели; гипепараметры подобраны при помощи четырёх методов
Обучить две модели; гипепараметры подобраны при помощи четырёх методов:
использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna),
использована кросс-валидация,
логистическая регрессия,
случайный лес.

В качестве метрики будем использовать F1-score.

In [1]:
#Импортируем используемые библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

import warnings # для игнорирования предупреждений
#Игнорируем предупреждения
warnings.filterwarnings('ignore')

In [2]:
# Загружаем датафрейм.
data_bio = pd.read_csv('C:\\Users\\ievge\\Desktop\\jupyter\\data\\_train_sem09 (1).csv')
data_bio

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,1,0.033300,0.506409,0.10,0.0,0.209887,0.633426,0.297659,0.376124,0.727093,...,0,0,0,0,0,0,0,0,0,0
3747,1,0.133333,0.651023,0.15,0.0,0.151154,0.766505,0.170876,0.404546,0.787935,...,0,0,1,0,1,0,1,0,0,0
3748,0,0.200000,0.520564,0.00,0.0,0.179949,0.768785,0.177341,0.471179,0.872241,...,0,0,0,0,0,0,0,0,0,0
3749,1,0.100000,0.765646,0.00,0.0,0.536954,0.634936,0.342713,0.447162,0.672689,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Отделяем из датасет целевой признак
y = data_bio['Activity']
x = data_bio.drop(['Activity'], axis=1)

# Разбиваем данные на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)

##### Логистическая регрессия

Посмотрим на метрику f1-score при устрановленных по умолчанию гиперпараметров.

In [4]:
#Обучаем модель
log_reg = linear_model.LogisticRegression(random_state=42, max_iter=600)
log_reg.fit(X_train, y_train)

#Делаем предсказание
log_reg_train_pred = log_reg.predict(X_train)
log_reg_test_pred = log_reg.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, log_reg_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(log_reg.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, log_reg_test_pred)))

f1_score на тренировочном наборе: 0.90
accuracy на тестовом наборе: 0.76
f1_score на тестовом наборе: 0.78


##### GridSearchCV

In [5]:
#Задаем поиск гиперпараметров в виде словаря
param_grid = {'penalty': ['l2'],
              'solver': ['lbfgs', 'sag'],
              'C': list(np.linspace(0.1, 0.5, 20, dtype=float))
              }

#С помощью GridSearchCV находим лучшие параметры для логистической регрессии
grid_search_lr = GridSearchCV(
    estimator=linear_model.LogisticRegression(
    random_state=42,
    max_iter=1000
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  

#Обучаем модель и предсказываем результаты
%time grid_search_lr.fit(X_train, y_train)
y_train_pred = grid_search_lr.predict(X_train)
y_test_pred = grid_search_lr.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_lr.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_lr.best_params_))


Wall time: 16min 44s
f1_score на тренировочном наборе: 0.85
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


##### Вывод:
С помощью гиперпараметров класса GridSearchCV нам удалось улучшить метрику f1-score, она составила - 0.80.

##### RandomizedSearchCV

In [6]:
#Задаем искомые гиперпараметры в виде словаря
param_distributions = {'penalty': ['l2'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.1, 0.5, 20, dtype=float))},

#С помощью RandomizedSearchCV находим параметры для логистической регрессии         
random_search_lr = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 15, 
    n_jobs = -1
)  

#Обучаем модель и предсказываем результаты
%time random_search_lr.fit(X_train, y_train)
y_train_pred = random_search_lr.predict(X_train)
y_test_pred = random_search_lr.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_lr.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_lr.best_params_))

Wall time: 7min 44s
f1_score на тренировочном наборе: 0.86
accuracy на тестовом наборе: 0.77
f1_score на тестовом наборе: 0.80
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.12105263157894737}


##### Вывод:
При использовании класса RandomizedSearchCV мы нашли такие же параметры как и в GridSearchCV, но значительно сократили время работы кода.

##### Hyperopt

In [7]:
#Задаем искомые гиперпараметры в виде словаря
penalties = ['l2']
solvers = ['sag', 'lbfgs']
space = {'penalty': hp.choice('penalty', penalties),
        'solver': hp.choice('solver', solvers),
        'C': hp.uniform('C',0,1)}


def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""

#Строим модель
    model = linear_model.LogisticRegression(**params, random_state=42, max_iter=5000)

#Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

#Метрику необходимо минимизировать, поэтому ставим знак минус
    return -score


#Залогируем полученные результаты
trials = Trials() #используется для логирования результатов

#Начинаем подбор гиперпараметров.
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=25,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 25/25 [11:43<00:00, 28.14s/trial, best loss: -0.7785419212939304]
Наилучшие значения гиперпараметров {'C': 0.09664464567700914, 'penalty': 0, 'solver': 0}


In [8]:
#Строим логистическую модель
model = linear_model.LogisticRegression(
    max_iter=1000,
    random_state=42, 
    penalty=penalties[best['penalty']],
    solver=solvers[best['solver']],
    C=best['C']
)

#Обучаем модель
model.fit(X_train, y_train)

#Предсказываем результат для тренировочной и тестовой выборки
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#Рассчитываем метрики
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

#Продолжим подбор наилучших гиперпараметров
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=30,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(best))

f1_score на обучающем наборе: 0.85
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80
100%|███████████████████████████████████████████████| 30/30 [03:07<00:00, 37.50s/trial, best loss: -0.7785419212939304]
Наилучшие значения гиперпараметров {'C': 0.09664464567700914, 'penalty': 0, 'solver': 0}


In [9]:
#Строим еще одну модель логистической регрессии
model = linear_model.LogisticRegression(
    max_iter=1000,
    random_state=42, 
    penalty=penalties[best['penalty']],
    solver=solvers[best['solver']],
    C=best['C']
)

#Обучаем модель
model.fit(X_train, y_train)

#Разбиваем на тренировочную и тестовую выборки
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#Рассчитываем метрики
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 0.85
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80


##### Вывод:
С помощью Hyperopt и использования кросс-валидации метрика f1-score составила - 0.80. Нам неудалось еще больше улучшить результаты.

##### Optuna

In [10]:
def optuna_rf(trial, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""
    
#Задаем пространства поиска гиперпараметров
    penalties = ['l2']
    solvers = ['sag', 'lbfgs']
    penalty = trial.suggest_categorical('penalty', penalties)
    solver = trial.suggest_categorical('solver', solvers)
    C = trial.suggest_float('C', 0, 1)

#Создаем модель
    model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=42,
                                          max_iter=1000)
#Обучаем модель
    model.fit(X_train, y_train)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return score


#Создаем исследуемый объект
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")

#Ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

print("Наилучшие значения гиперпараметров {}".format(study.best_params))

[I 2023-07-01 17:42:54,994] A new study created in memory with name: LogisticRegression
[I 2023-07-01 17:43:37,534] Trial 0 finished with value: 0.7779636562410126 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.06617248288477207}. Best is trial 0 with value: 0.7779636562410126.
[I 2023-07-01 17:44:33,689] Trial 1 finished with value: 0.7736781571834817 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.9040635567315681}. Best is trial 0 with value: 0.7779636562410126.
[I 2023-07-01 17:44:46,359] Trial 2 finished with value: 0.7716018385912831 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.7795880121993686}. Best is trial 0 with value: 0.7779636562410126.
[I 2023-07-01 17:44:50,454] Trial 3 finished with value: 0.7827771905495385 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.03192810231452914}. Best is trial 3 with value: 0.7827771905495385.
[I 2023-07-01 17:45:54,110] Trial 4 finished with value: 0.7749393407438795 and parameters: {'penalty'

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.03192810231452914}


In [11]:
#Создаем модель
model = linear_model.LogisticRegression(**study.best_params,random_state=42, max_iter=1000)

#Обучаем модель
model.fit(X_train, y_train)

#Предсказываем результат для тренировочной и тестовой выборки
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 0.83
accuracy на тестовом наборе: 0.78
f1_score на тестовом наборе: 0.80


Вывод:
С помощью гиперпараметров найденных Optuna получили такие же значения метрики f1-score как и при других классах 0.80.

2. Случайный лес
Посмотрим на метрику f1-score при устрановленных по умолчанию гиперпараметров.

In [12]:
#Обучаем модель
forest = ensemble.RandomForestClassifier(random_state=42)
forest.fit(X_train, y_train)

#Делаем предсказание
forest_train_pred = forest.predict(X_train)
forest_test_pred = forest.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, forest_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(forest.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, forest_test_pred)))

f1_score на тренировочном наборе: 1.00
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


##### GridSearchCV

In [13]:
#Задаем искомые гиперпараметры в виде словаря
param_grid = {'n_estimators': list(range(80, 200, 30)),
            'min_samples_leaf': [3],
            'max_depth': list(np.linspace(20, 40, 5, dtype=int)),
            'criterion': ['gini', 'entropy']
            }

#С помощью GridSearchCV находим наилучшие параметры для логистической регрессии
grid_search_forest = GridSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  

#Обучаем модель и предсказываем результаты
%time grid_search_forest.fit(X_train, y_train)
y_train_pred = grid_search_forest.predict(X_train)
y_test_pred = grid_search_forest.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_forest.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_forest.best_params_))

Wall time: 2min 31s
f1_score на тренировочном наборе: 0.98
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 3, 'n_estimators': 110}


Вывод:
С помощью подобранных гипераметров классом GridSearchCV нам не удалось улучшить метрику f1-score и она также составила 0.82.



##### RandomizedSearchCV

In [14]:
#Задаем искомые гиперпараметры в виде словаря
param_distributions = {'n_estimators': list(range(80, 200, 30)),
            'min_samples_leaf': [3],
            'max_depth': list(np.linspace(20, 40, 5, dtype=int)),
            'criterion': ['gini', 'entropy']
            }

#С помощью RandomizedSearchCV находим наилучшие параметры для логистической регрессии            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 15, 
    n_jobs = -1
)  

#Обучаем модель и предсказываем результаты
%time random_search_forest.fit(X_train, y_train)
y_train_pred = random_search_forest.predict(X_train)
y_test_pred = random_search_forest.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(random_search_forest.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

Wall time: 52.8 s
f1_score на тренировочном наборе: 0.98
accuracy на тестовом наборе: 0.79
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 110, 'min_samples_leaf': 3, 'max_depth': 40, 'criterion': 'entropy'}


##### Вывод:
С помощью подобранных гиперпараметров классом RandomizedSearchCV метрика f1-score осталась такая же как и при помощи GridSearchCV.

##### Hyperopt

In [15]:
#Задаем искомые гиперпараметры в виде словаря
criterions = ['gini', 'entropy']
space={'n_estimators': hp.quniform('n_estimators', 80, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 40, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1),
       'criterion': hp.choice('criterion', criterions)
      }


def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""
    params = {'n_estimators': int(params['n_estimators']), 
            'max_depth': int(params['max_depth']), 
            'min_samples_leaf': int(params['min_samples_leaf']),
            'criterion': params['criterion']
            }
#Строим модель.
    model = ensemble.RandomForestClassifier(**params, random_state=42)

#Обучаем модель с помощью кросс-валидации
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

#Метрику необходимо минимизировать, поэтому ставим знак минус.
    return -score


#Залогируем полученные результаты
trials = Trials() # используется для логирования результатов

#Начинаем подбор гиперпараметров
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=25,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 25/25 [02:04<00:00,  4.97s/trial, best loss: -0.8127261258298031]
Наилучшие значения гиперпараметров {'criterion': 1, 'max_depth': 22.0, 'min_samples_leaf': 2.0, 'n_estimators': 154.0}


In [16]:
#Строим  модель
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    criterion=criterions[best['criterion']],
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

#Обучаем модель
model.fit(X_train, y_train)

#Предсказываем результат для тренировочной и тестовой выборки
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#Рассчитываем метрики
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 1.00
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


In [17]:
#Продолжим подбор наилучших гиперпараметров
best = fmin(hyperopt_rf,
          space=space,
          algo=tpe.suggest,
          max_evals=30,
          trials=trials,
          rstate=np.random.default_rng(42)
         )

#Выводим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 30/30 [00:33<00:00,  6.72s/trial, best loss: -0.8127261258298031]
Наилучшие значения гиперпараметров {'criterion': 1, 'max_depth': 22.0, 'min_samples_leaf': 2.0, 'n_estimators': 154.0}


In [18]:
#Строим модель
model = ensemble.RandomForestClassifier(
    random_state=42, 
    n_estimators=int(best['n_estimators']),
    criterion=criterions[best['criterion']],
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)

#Обучаем модель
model.fit(X_train, y_train)

#Предсказываем результат для тренировочной и тестовой выборки
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#Рассчитываем метрики
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на обучающем наборе: 1.00
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


##### Вывод:
С помощью Hyperopt мы не смогли улучшить f1-score.

##### Optuna

In [19]:
def optuna_rf(trial, cv=5, X=X_train, y=y_train, random_state=42):
    """Функция получения комбинации гиперпараметров."""
    
#Задаем пространства поиска гиперпараметров
    criterions = ['gini', 'entropy']
    n_estimators = trial.suggest_int('n_estimators', 50, 200, 1)
    max_depth = trial.suggest_int('max_depth', 5, 40, 1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
    criterion = trial.suggest_categorical('criterion', criterions)

#Создаем модель
    model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        min_samples_leaf=min_samples_leaf,
                                        criterion=criterion,
                                        random_state=42)
#Обучаем модель
    model.fit(X_train, y_train)
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return score


#Создаем объект исследования
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")

#Ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=30)

print("Наилучшие значения гиперпараметров {}".format(study.best_params))

[I 2023-07-01 17:56:50,356] A new study created in memory with name: RandomForestClassifier
[I 2023-07-01 17:56:54,498] Trial 0 finished with value: 0.7828369996861662 and parameters: {'n_estimators': 99, 'max_depth': 8, 'min_samples_leaf': 7, 'criterion': 'entropy'}. Best is trial 0 with value: 0.7828369996861662.
[I 2023-07-01 17:56:56,837] Trial 1 finished with value: 0.7854776038813048 and parameters: {'n_estimators': 50, 'max_depth': 31, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 1 with value: 0.7854776038813048.
[I 2023-07-01 17:57:00,429] Trial 2 finished with value: 0.7881462648056984 and parameters: {'n_estimators': 85, 'max_depth': 14, 'min_samples_leaf': 9, 'criterion': 'gini'}. Best is trial 2 with value: 0.7881462648056984.
[I 2023-07-01 17:57:04,563] Trial 3 finished with value: 0.8027928988271249 and parameters: {'n_estimators': 66, 'max_depth': 39, 'min_samples_leaf': 3, 'criterion': 'entropy'}. Best is trial 3 with value: 0.8027928988271249.
[I 2023-07

Наилучшие значения гиперпараметров {'n_estimators': 187, 'max_depth': 33, 'min_samples_leaf': 2, 'criterion': 'entropy'}


In [20]:
#Создаем модель
model = ensemble.RandomForestClassifier(**study.best_params,random_state=42)

#Обучаем модель
model.fit(X_train, y_train)

#Предсказываем результат для тренировочной и тестовой выборки
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

#Рассчитываем метрики
print('f1_score на тренировочном наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print("accuracy на тестовом наборе: {:.2f}".format(model.score(X_test, y_test)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тренировочном наборе: 1.00
accuracy на тестовом наборе: 0.80
f1_score на тестовом наборе: 0.82


# Вывод:
С помощью Optuna не удалось еще больше учучшить f1-score.

Наилучший f1-score мы получили при использовании модели случайного леса с базовыми гиперпараметрами и она составила 0.82, также помощью Hyperopt и использования кросс-валидации метрика f1-score составила - 0.80 Все подобранные разными методами гиперпараметры не смогли улучшить метрику.